In [1]:
# import functions
import sys,os
import glob
try: 
    if(cwd is not None):
        from functions.utils_binary import *
except:
    %cd ..
    cwd = os.getcwd()
    sys.path.insert(0,cwd)
    from functions.utils_binary import *

d:\cheminformatics_pipeline


In [ ]:
# prepare data for training
defineXY(testSize=0.2, trainSize=0.8, randomState=4)

## XGBoost, Ada and LightGBM pipeline

In [ ]:
# setting search space

xgb_search = {
'learning_rate': Real(0.01, 1.0, 'uniform'),
'max_depth': Integer(2, 12),
'subsample': Real(0.1, 1.0, 'uniform'),
'colsample_bytree': Real(0.1, 1.0, 'uniform'),
'reg_alpha': Real(1e-9, 100., 'uniform'),
'n_estimators': Integer(50, 5000)
}

ada_search = {
    'learning_rate': Real(0.005, 0.9, prior="log-uniform"),
    'n_estimators': Integer(1, 1000),
}

lgbm_search = {
    'reg_sqrt': Categorical([True, False]),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),     # Boosting learning rate
    'n_estimators': Integer(30, 5000),                   # Number of boosted trees to fit
    'num_leaves': Integer(2, 512),                       # Maximum tree leaves for base learners
    'max_depth': Integer(-1, 256),                       # Maximum tree depth for base learners, <=0 means no limit
    'subsample': Real(0.01, 1.0, 'uniform'),             # Subsample ratio of the training instance
    'subsample_freq': Integer(1, 10),                    # Frequency of subsample, <=0 means no enable
    'colsample_bytree': Real(0.01, 1.0, 'uniform'),      # Subsample ratio of columns when constructing each tree
    'reg_lambda': Real(1e-9, 100.0, 'log-uniform'),      # L2 regularization
    'reg_alpha': Real(1e-9, 100.0, 'log-uniform'),       # L1 regularization
   }

In [ ]:
# scorer
f1_scorer = make_scorer(f1_score, pos_label=1)

# Setting the validation strategy
inner_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
outer_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [ ]:
# initiate models
init_models = [xgb.XGBClassifier(), LGBMClassifier(), AdaBoostClassifier()]

# search spaces
search_spaces = [xgb_search, lgbm_search, ada_search]

In [ ]:
# define fp generated
fp_generated = ['ecfp_fp_2_1024', 'fcfp_fp_2_1024', 'maccs_fp']

for fp in fp_generated:

    # run through x train
    for file in os.listdir('./data/curated_data/x/binary/x_train/'):

        # run through y train
        for file_ in os.listdir('./data/curated_data/y/binary/y_train/'):

            # check if file is json file
            if file.endswith('json'):

                #check if files share the same fp, if so, continue
                if file.find(fp) != -1 and  file_.find(fp) != -1:
                    
                    # load x_train
                    with open(f'./data/curated_data/x/binary/x_train/{file}', 'r') as x_train:
                        x_train = json.load(x_train)
                        x_train= np.asarray(x_train)


                    # load y_train
                    with open(f'./data/curated_data/y/binary/y_train/{file_}', 'r') as y_train:
                        y_train = json.load(y_train)
                        y_train= np.asarray(y_train)

                    # training
                    bayessearch(
                        models = init_models, 
                        search_spaces = search_spaces, 
                        scorer = f1_scorer,
                        inner_cv = inner_skf,
                        outer_cv = outer_skf,
                        x = x_train,
                        y = y_train,
                        fp = fp
                    )